### DATASET DOWNLOAD

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from fastai.vision.all import *


Google_Image = 'https://www.google.com/search?site=&tbm=isch&source=hp&biw=1873&bih=990&'

u_agnt = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive',
}

Image_folder = "bears"

def main():
    if not os.path.exists(Image_folder):
        os.mkdir(Image_folder)
    download_images()


def download_images():
    datas = []
    while True:
            data = input("Enter the Search keyword or 'q' to quit")
            if data == 'q':
                break
            datas.append(data)
    
    num_images = int(input("Enter the number of images to be downloaded for each type of image"))
    
    print(f"Searching images for...{datas}")
    
    for data in datas:
        if " " in data:
            new_data = data.split(" ")
            search_url = Google_Image + 'q=' + new_data[0] +"+" + new_data[1] 
    
        search_url = Google_Image + 'q=' + data 
            
        response = requests.get(search_url, headers=u_agnt)
        html = response.text
        b_soup = BeautifulSoup(html, 'html.parser')
        results = b_soup.findAll('img', {'class': 'rg_i Q4LuWd'})
        
         # extract the links of requested number of images with 'data-src' attribute and appended those links to a list 'imagelinks'
        # allow to continue the loop in case query fails for non-data-src attributes

        count = 0
        imagelinks = []
        for res in results:
            try:
                link = res['data-src']
                imagelinks.append(link)
                count = count + 1
                if (count >= num_images):
                    break

            except KeyError:
                continue
                
            print(f'Found {len(imagelinks)} images of ' + data)

        # Downloading images
        print(f'Start downloading images of ' + data + ' ...')

        # Creating a subfolder in the main folder
        Image_Subfolder = data
        if not os.path.exists(os.path.join(Image_folder, Image_Subfolder)):
            os.mkdir(os.path.join(Image_folder, Image_Subfolder))

        for i, imagelink in enumerate(imagelinks):
            # open each image link and save the file
            response = requests.get(imagelink)
            imagename = Image_folder + '/' + Image_Subfolder + '/' + data + str(i + 1) + '.jpg'
            with open(imagename, 'wb') as file:
                file.write(response.content)

        print(f'Downloading ' + data + ' is completed!\n')
    return response

        
if __name__ == '__main__':
    main()


### DATASET RENAMING

In [ ]:
from pathlib import Path

bear_files = Path("bears/grizzly_bear")

for file in bear_files.iterdir():
    directory = file.parent
    old_name = file.stem
    new_name = f"{old_name}.jpg"
    
    file.rename(Path(directory, new_name))